# pandas——groupby操作
实验目的：熟练掌握pandas中的groupby操作

groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False）

参数说明：

by是指分组依据（列表、字典、函数，元组，Series）

axis：是作用维度（0为行，1为列）

level：根据索引级别分组

sort：对groupby分组后新的dataframe中索引进行排序，sort=True为升序，

as_index：在groupby中使用的键是否成为新的dataframe中的索引，默认as_index=True

group_keys：在调用apply时，将group键添加到索引中以识别片段

squeeze ：如果可能的话，减少返回类型的维数，否则返回一个一致的类型

grouping操作（split-apply-combine）

数据的分组&聚合 -- 什么是groupby 技术?

在数据分析中，我们往往需要在将数据拆分，在每一个特定的组里进行运算。比如根据教育水平和年龄段计算某个城市的工作人口的平均收入。

pandas中的groupby提供了一个高效的数据的分组运算。

我们通过一个或者多个分类变量将数据拆分，然后分别在拆分以后的数据上进行需要的计算

我们可以把上述过程理解为三部：

1.拆分数据（split） 

2.应用某个函数（apply） 

3.汇总计算结果（aggregate）

下面这个演示图展示了“分拆-应用-汇总”的groupby思想

“分拆-应用-汇总”的groupby思想

分解步骤：

Step1 ：数据分组—— groupby 方法

Step2 ：数据聚合：

     使用内置函数——sum / mean / max / min / count等

     使用自定义函数—— agg ( aggregate ) 方法

     自定义更丰富的分组运算—— apply 方法


In [2]:
# 创建一个数据帧df。
import numpy as np  
import pandas as pd  
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],'C' : np.random.randn(8),'D' : np.random.randn(8)})  
print(df)  

A      B         C         D
0  foo    one -0.226287 -1.593580
1  bar    one  0.812658 -0.308767
2  foo    two -0.785831  0.976460
3  bar  three  0.479241 -0.043474
4  foo    two  1.770236 -0.211644
5  bar    two  0.644075 -0.896903
6  foo    one -0.174470  0.864171
7  foo  three -0.303078 -0.180676


## 分组

In [3]:
# 通过A列对df进行分布操作。
df.groupby('A')  

In [4]:
# 通过A、B列对df进行分组操作。
df.groupby(['A','B']) 

In [5]:
# 使用自定义函数进行分组操作，自定义一个函数，使用groupby方法并使用自定义函数给定的条件，按列对df进行分组。
def get_letter_type(letter):  
    if letter.lower() in 'aeiou':  
        return 'vowel'  
    else:  
        return 'consonant'  
  
grouped = df.groupby(get_letter_type, axis=1)  
for group in grouped:  
    print(group)  

('consonant',        B         C         D
0    one -0.226287 -1.593580
1    one  0.812658 -0.308767
2    two -0.785831  0.976460
3  three  0.479241 -0.043474
4    two  1.770236 -0.211644
5    two  0.644075 -0.896903
6    one -0.174470  0.864171
7  three -0.303078 -0.180676)
('vowel',      A
0  foo
1  bar
2  foo
3  bar
4  foo
5  bar
6  foo
7  foo)


In [8]:
# 创建一个Series名为s，使用groupby根据s的索引对s进行分组，返回分组后的新Series，对新Series进行first、last、sum操作。
lst = [1, 2, 3, 1, 2, 3]  
s = pd.Series([1, 2, 3, 10, 20, 30], lst)  
grouped = s.groupby(level=0)  
#查看分组后的第一行数据 
grouped.first()   

1    1
2    2
3    3
dtype: int64

In [9]:
#查看分组后的最后一行数据  
grouped.last()  

1    10
2    20
3    30
dtype: int64

In [10]:
#对分组的各组进行求和  
grouped.sum()  

1    11
2    22
3    33
dtype: int64

In [11]:
# 分组排序，使用groupby进行分组时，默认是按分组后索引进行升序排列，在groupby方法中加入sort=False参数，可以进行降序排列。
df2=pd.DataFrame({'X':['B','B','A','A'],'Y':[1,2,3,4]})  
#按X列对df2进行分组，并求每组的和  
df2.groupby(['X']).sum()  

,Y
X,
A,7
B,3


In [12]:
#按X列对df2进行分组，分组时不对键进行排序，并求每组的和  
df2.groupby(['X'],sort=False).sum()  

,Y
X,
B,3
A,7


In [13]:
# 使用get_group方法得到分组后某组的值。
df3 = pd.DataFrame({'X' : ['A', 'B', 'A', 'B'], 'Y' : [1, 4, 3, 2]})  
#按X列df3进行分组，并得到A组的df3值  
df3.groupby(['X']).get_group('A')  

,X,Y
0,A,1
2,A,3


In [14]:
#按X列df3进行分组，并得到B组的df3值  
df3.groupby(['X']).get_group('B') 

,X,Y
1,B,4
3,B,2


In [15]:
# 使用groups方法得到分组后所有组的值。
df.groupby('A').groups  

{'bar': Int64Index([1, 3, 5], dtype='int64'),
 'foo': Int64Index([0, 2, 4, 6, 7], dtype='int64')}

In [16]:
df.groupby(['A','B']).groups  

{('bar', 'one'): Int64Index([1], dtype='int64'),
 ('bar', 'three'): Int64Index([3], dtype='int64'),
 ('bar', 'two'): Int64Index([5], dtype='int64'),
 ('foo', 'one'): Int64Index([0, 6], dtype='int64'),
 ('foo', 'three'): Int64Index([7], dtype='int64'),
 ('foo', 'two'): Int64Index([2, 4], dtype='int64')}

查看分组对象的所有内置函数

In [18]:
# 使用groupby方法按A列对df进行分组操作，将结果赋值给grouped。
grouped=df.groupby(['A']) 
grouped 

分组对象的所有的内置函数如下所示：

grouped.agg        grouped.boxplot    grouped.cummin     grouped.describe   grouped.filter     grouped.get_group  grouped.height     grouped.last       grouped.median     grouped.ngroups    grouped.plot       grouped.rank       grouped.std        grouped.transform  

grouped.aggregate  grouped.count      grouped.cumprod    grouped.dtype      grouped.first      grouped.groups     grouped.hist       grouped.max        grouped.min        grouped.nth        grouped.prod       grouped.resample   grouped.sum        grouped.var  

grouped.apply      grouped.cummax     grouped.cumsum     grouped.fillna     grouped.gender     grouped.head       grouped.indices    grouped.mean       grouped.name       grouped.ohlc       grouped.quantile   grouped.size       grouped.tail       grouped.weight  

In [20]:
# 多级索引分组，创建一个有两级索引的Series，并使用两个方法对Series进行分组并求和。
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]  
index=pd.MultiIndex.from_arrays(arrays,names=['first','second'])  
s=pd.Series(np.random.randn(8),index=index)  
s.groupby(level=0).sum()  

first
bar   -2.611396
baz    1.048773
foo    0.861719
qux   -3.110721
dtype: float64

In [21]:
s.groupby(level='second').sum()  

second
one   -1.360378
two   -2.451247
dtype: float64

In [22]:
# 复合分组，对s按first、second进行分组并求和。
s.groupby(level=['first', 'second']).sum()

first  second
bar    one      -1.377482
       two      -1.233914
baz    one       0.361194
       two       0.687579
foo    one       1.199412
       two      -0.337693
qux    one      -1.543502
       two      -1.567219
dtype: float64

In [24]:
# 复合分组（按索引和列），创建数据帧df，使用索引级别和列对df进行分组。
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]  
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])  
df = pd.DataFrame({'A': [1, 1, 1, 1, 2, 2, 3, 3], 'B': np.arange(8)},index=index)  
print(df)  

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7


In [25]:
df.groupby([pd.Grouper(level=1),'A']).sum()  

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [26]:
# 对df进行分组，将分组后C列的值赋值给grouped，统计grouped中每类的个数。
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],'C' : np.random.randn(8),'D' : np.random.randn(8)})  
grouped=df.groupby(['A'])  
grouped_C=grouped['C']  
print(grouped_C.count())  

A
bar    3
foo    5
Name: C, dtype: int64


In [27]:
# 对上面创建的df的C列，按A列值进行分组并求和。
df['C'].groupby(df['A']).sum()  

A
bar    0.280014
foo   -3.178421
Name: C, dtype: float64

In [28]:
# 遍历分组结果，通过A，B两列对df进行分组，分组结果的组名为元组。
for name, group in df.groupby(['A', 'B']):  
    print(name)  
    print(group)  

('bar', 'one')
     A    B         C         D
1  bar  one -1.080454 -1.011269
('bar', 'three')
     A      B         C         D
3  bar  three -0.674847 -0.721901
('bar', 'two')
     A    B         C         D
5  bar  two  2.035315 -2.670425
('foo', 'one')
     A    B         C         D
0  foo  one -0.278042  0.492636
6  foo  one -0.333590  1.869623
('foo', 'three')
     A      B         C         D
7  foo  three  0.204505 -0.414279
('foo', 'two')
     A    B         C         D
2  foo  two -1.630711 -1.583617
4  foo  two -1.140583 -1.031174


In [29]:
# 通过A列对df进行分组，并查看分组对象的bar列。
df.groupby(['A']).get_group(('bar'))  

,A,B,C,D
1,bar,one,-1.080454,-1.011269
3,bar,three,-0.674847,-0.721901
5,bar,two,2.035315,-2.670425


In [30]:
# 按A,B两列对df进行分组，并查看分组对象中bar、one都存在的部分。
df.groupby(['A','B']).get_group(('bar','one'))  

,A,B,C,D
1,bar,one,-1.080454,-1.011269


**注意:当分组按两列来分时，查看分组对象也应该包含每列的一部分。**

## 聚集

In [31]:
# 聚合操作，按A列对df进行分组，使用聚合函数aggregate求每组的和。
grouped=df.groupby(['A'])  
grouped.aggregate(np.sum)  

,C,D
A,,
bar,0.280014,-4.403594
foo,-3.178421,-0.666810


In [33]:
# 按A、B两列对df进行分组，并使用聚合函数aggregate对每组求和。
grouped=df.groupby(['A','B'])  
grouped.aggregate(np.sum)  

C         D
A   B                        
bar one   -1.080454 -1.011269
    three -0.674847 -0.721901
    two    2.035315 -2.670425
foo one   -0.611632  2.362259
    three  0.204505 -0.414279
    two   -2.771294 -2.614790

**注意：通过上面的结果可以看到。聚合完成后每组都有一个组名作为新的索引，使用as_index=False可以忽略组名。**

In [34]:
# 当as_index=True时，在groupby中使用的键将成为新的dataframe中的索引。按A、B两列对df进行分组，这是使参数as_index=False，再使用聚合函数aggregate求每组的和。
grouped=df.groupby(['A','B'],as_index=False)  
grouped.aggregate(np.sum)  

,A,B,C,D
0,bar,one,-1.080454,-1.011269
1,bar,three,-0.674847,-0.721901
2,bar,two,2.035315,-2.670425
3,foo,one,-0.611632,2.362259
4,foo,three,0.204505,-0.414279
5,foo,two,-2.771294,-2.614790


In [35]:
# 使用reset_index函数可以得到与参数as_index=False相同的结果。
df.groupby(['A','B']).sum().reset_index()  

,A,B,C,D
0,bar,one,-1.080454,-1.011269
1,bar,three,-0.674847,-0.721901
2,bar,two,2.035315,-2.670425
3,foo,one,-0.611632,2.362259
4,foo,three,0.204505,-0.414279
5,foo,two,-2.771294,-2.614790


In [36]:
# 聚合操作，按A、B列对df进行分组，使用size方法，求每组的大小。返回一个Series，索引是组名，值是每组的大小。
grouped=df.groupby(['A','B'])  
grouped.size()  

A    B    
bar  one      1
     three    1
     two      1
foo  one      2
     three    1
     two      2
dtype: int64

In [37]:
# 聚合操作，对分组grouped进行统计描述。
grouped.describe()  

C                                                              \
          count      mean       std       min       25%       50%       75%   
A   B                                                                         
bar one     1.0 -1.080454       NaN -1.080454 -1.080454 -1.080454 -1.080454   
    three   1.0 -0.674847       NaN -0.674847 -0.674847 -0.674847 -0.674847   
    two     1.0  2.035315       NaN  2.035315  2.035315  2.035315  2.035315   
foo one     2.0 -0.305816  0.039279 -0.333590 -0.319703 -0.305816 -0.291929   
    three   1.0  0.204505       NaN  0.204505  0.204505  0.204505  0.204505   
    two     2.0 -1.385647  0.346574 -1.630711 -1.508179 -1.385647 -1.263115   

                        D                                                    \
                max count      mean       std       min       25%       50%   
A   B                                                                         
bar one   -1.080454   1.0 -1.011269       NaN -1.011269 -1.011269 -1.011269   
    three -0.674847   1.0 -0.721901       NaN -0.721901 -0.721901 -0.721901   
    two    2.035315   1.0 -2.670425       NaN -2.670425 -2.670425 -2.670425   
foo one   -0.278042   2.0  1.181130  0.973677  0.492636  0.836883  1.181130   
    three  0.204505   1.0 -0.414279       NaN -0.414279 -0.414279 -0.414279   
    two   -1.140583   2.0 -1.307395  0.390636 -1.583617 -1.445506 -1.307395   

                               
                75%       max  
A   B                          
bar one   -1.011269 -1.011269  
    three -0.721901 -0.721901  
    two   -2.670425 -2.670425  
foo one    1.525377  1.869623  
    three -0.414279 -0.414279  
    two   -1.169284 -1.031174

**注意：聚合函数可以减少数据帧的维度，常用的聚合函数有：mean、sum、size、count、std、var、sem 、describe、first、last、nth、min、max。**

执行多个函数在一个分组结果上：在分组返回的Series中我们可以通过一个聚合函数的列表或一个字典去操作series，返回一个DataFrame。

In [39]:
# 按A列对df进行分组，再对分组结果grouped的C列进行sum、mean、std三个聚合操作。
grouped=df.groupby('A')  
grouped['C'].agg([np.sum,np.mean,np.std])  

,sum,mean,std
A,,,
bar,0.280014,0.093338,1.693985
foo,-3.178421,-0.635684,0.736561


In [40]:
# 对分组结果grouped的C列进行sum、mean、std三个聚合操作，然后使用rename方法修改返回结果中的列名，将sum.mean,std分别替换为：foo,bar，baz。
grouped['C'].agg([np.sum,np.mean,np.std]).rename(columns={'sum':'foo','mean':'bar','std':'baz'})  

,foo,bar,baz
A,,,
bar,0.280014,0.093338,1.693985
foo,-3.178421,-0.635684,0.736561


作用不同的聚合函数到DataFrame的不同列上，通过聚合函数的一个字典作用不同的聚合函数到一个DataFrame的列上。

In [41]:
# 对分组结果grouped进行聚合操作，C列上作用sum函数，D列上作用std函数并使参数ddof=1。
grouped.agg({'C':np.sum,'D':lambda x:np.std(x,ddof=1)})  

,C,D
A,,
bar,0.280014,1.051449
foo,-3.178421,1.359416


In [42]:
grouped.agg({'C':np.sum,'D':np.std})  

,C,D
A,,
bar,0.280014,1.051449
foo,-3.178421,1.359416


In [43]:
# transform操作:对数组在分组内进行转换，而不是全局转换。
index = pd.date_range('10/1/1999', periods=1100)  
ts = pd.Series(np.random.normal(0.5, 2, 1100), index)  
ts.head()  

1999-10-01   -0.634503
1999-10-02    4.404585
1999-10-03   -0.560073
1999-10-04    1.385955
1999-10-05   -0.959831
Freq: D, dtype: float64

In [44]:
ts.tail()

2002-09-30    0.027739
2002-10-01    1.027486
2002-10-02   -2.224909
2002-10-03   -0.679539
2002-10-04    3.290294
Freq: D, dtype: float64

In [45]:
key=lambda x:x.year  
zscore=lambda x:(x-x.mean())/x.std()  
ts.groupby(key).transform(zscore)  

1999-10-01   -0.669370
1999-10-02    1.953299
1999-10-03   -0.630632
1999-10-04    0.382208
1999-10-05   -0.838692
                ...   
2002-09-30   -0.235584
2002-10-01    0.267135
2002-10-02   -1.368319
2002-10-03   -0.591236
2002-10-04    1.404979
Freq: D, Length: 1100, dtype: float64

filter操作：filter方法返回的是原对象的一个子集。

In [46]:
# 创建一个Series值为[1,1,2,3,3,3]并命名为sf，对sf按sf进行分组，然后过滤出每组的和大于2的值。
sf=pd.Series([1,1,2,2,3,3,3])  
sf.groupby(sf).filter(lambda x:x.sum()>2)  

2    2
3    2
4    3
5    3
6    3
dtype: int64